In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

In [2]:
bcf = pd.read_excel("BCF.xlsx")
ps_bart = pd.read_excel("ps_BART.xlsx")

In [3]:
bcf.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [4]:
# Calculate the mean and standard deviation for each column
mean_values = bcf.mean()
std_values = bcf.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        271.529356
empirical_ATE_MAE         143.445463
empirical_ATE_MAPE         17.466232
empirical_ATE_len         227.385532
empirical_ATE_cover         0.655920
theoretical_CATE_RMSE     864.452989
theoretical_CATE_MAE      562.693816
theoretical_CATE_MAPE      79.303634
theoretical_CATE_len      318.292657
theoretical_CATE_cover      0.344095
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE         68.304788
empirical_ATE_MAE          27.049322
empirical_ATE_MAPE         65.808789
empirical_ATE_len          46.730033
empirical_ATE_cover         0.124481
theoretical_CATE_RMSE     605.907483
theoretical_CATE_MAE      362.449208
theoretical_CATE_MAPE     110.828665
theoretical_CATE_len       55.552066
theoretical_CATE_cover      0.040254
dtype: float64


In [5]:
bcf.head()

,empirical_ATE_RMSE,empirical_ATE_MAE,empirical_ATE_MAPE,empirical_ATE_len,empirical_ATE_cover,theoretical_CATE_RMSE,theoretical_CATE_MAE,theoretical_CATE_MAPE,theoretical_CATE_len,theoretical_CATE_cover
0,231.617220,123.085713,4.275952,232.438343,0.802,542.229526,371.368390,61.103882,253.822405,0.332824
1,311.411032,229.224170,10.479046,271.410150,0.398,479.236516,344.002519,94.171346,329.426108,0.400016
2,233.520672,125.103329,7.028941,178.015330,0.656,488.233054,333.897812,48.100952,272.338456,0.370624
3,544.276150,158.671931,9.241078,337.899576,0.766,3965.503230,2435.460699,51.059616,508.308548,0.232752
4,221.996374,114.890756,9.492876,164.346514,0.748,650.677316,423.520269,53.897625,240.638222,0.325040


In [6]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
bcf_SE_empirical_ATE_cover = (((0.95 - bcf['empirical_ATE_cover'])**2))
bcf_AE_empirical_ATE_cover = (np.abs(0.95 - bcf['empirical_ATE_cover']))

bcf_SE_theoretical_CATE_cover = (((0.95 - bcf['theoretical_CATE_cover'])**2))
bcf_AE_theoretical_CATE_cover = (np.abs(0.95 - bcf['theoretical_CATE_cover']))

In [7]:
ps_bart.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [8]:
# Calculate the mean and standard deviation for each column
mean_values = ps_bart.mean()
std_values = ps_bart.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE         18.850029
empirical_ATE_MAE          10.275159
empirical_ATE_MAPE          0.230697
empirical_ATE_len          43.891577
empirical_ATE_cover         0.862500
theoretical_CATE_RMSE      72.517558
theoretical_CATE_MAE       40.302486
theoretical_CATE_MAPE       1.531713
theoretical_CATE_len      192.095616
theoretical_CATE_cover      0.884291
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE         13.136283
empirical_ATE_MAE           3.223670
empirical_ATE_MAPE          0.476204
empirical_ATE_len          20.788895
empirical_ATE_cover         0.049537
theoretical_CATE_RMSE     107.709991
theoretical_CATE_MAE       59.417654
theoretical_CATE_MAPE       1.818976
theoretical_CATE_len      195.732600
theoretical_CATE_cover      0.038178
dtype: float64


In [9]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
ps_bart_SE_empirical_ATE_cover = (((0.95 - ps_bart['empirical_ATE_cover'])**2))
ps_bart_AE_empirical_ATE_cover = (np.abs(0.95 - ps_bart['empirical_ATE_cover']))

ps_bart_SE_theoretical_CATE_cover = (((0.95 - ps_bart['theoretical_CATE_cover'])**2))
ps_bart_AE_theoretical_CATE_cover = (np.abs(0.95 - ps_bart['theoretical_CATE_cover']))

In [10]:
# Function to check normality using Shapiro-Wilk test
def check_normality(data):
    stat, p = stats.shapiro(data)
    return p > 0.05  # If p > 0.05, data is normally distributed

# Function to check homoscedasticity using Levene's test
def check_homoscedasticity(data1, data2):
    stat, p = stats.levene(data1, data2)
    return p > 0.05  # If p > 0.05, variances are equal



# ATE RMSE

In [11]:
i=0
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_RMSE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 50.946035072408996, p-value: 1.7576255710528845e-11
Brown-Forsythe Test:
Statistic: 50.946035072408996, p-value: 1.7576255710528845e-11


# ATE MAE

In [12]:
i=1
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 53.338022782221444, p-value: 6.702574349514508e-12
Brown-Forsythe Test:
Statistic: 53.338022782221444, p-value: 6.702574349514508e-12


# ATE MAPE

In [13]:
i=2
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAPE
Mann-Whitney U Test:
Statistic: 9.0, p-value: 3.3568758082015105e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.71690149253732, p-value: 3.306909542149198e-34
Levene's Test:
Statistic: 3.5088871910018833, p-value: 0.06251402692826588
Brown-Forsythe Test:
Statistic: 3.5088871910018833, p-value: 0.06251402692826588


# CATE RMSE

In [14]:
i=5
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_RMSE
Mann-Whitney U Test:
Statistic: 120.0, p-value: 9.03384298505838e-33
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 142.17552238805968, p-value: 8.902306208837165e-33
Levene's Test:
Statistic: 25.175951776100344, p-value: 1.1614011093977905e-06
Brown-Forsythe Test:
Statistic: 25.175951776100344, p-value: 1.1614011093977905e-06


# CATE MAE

In [15]:
i=6
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAE
Mann-Whitney U Test:
Statistic: 71.0, p-value: 2.130842673676122e-33
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 145.0450208955225, p-value: 2.0995115948504247e-33
Levene's Test:
Statistic: 26.044789578068098, p-value: 7.79622338776249e-07
Brown-Forsythe Test:
Statistic: 26.044789578068098, p-value: 7.79622338776249e-07


# CATE MAPE

In [16]:
i=7
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAPE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 10.066380558910385, p-value: 0.0017506452712741838
Brown-Forsythe Test:
Statistic: 10.066380558910385, p-value: 0.0017506452712741838


# ATE len

In [17]:
i=3
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_len
Mann-Whitney U Test:
Statistic: 55.0, p-value: 1.3254780500722928e-33
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 145.98820895522385, p-value: 1.3059267761214299e-33
Levene's Test:
Statistic: 64.17188598865032, p-value: 9.59502304853687e-14
Brown-Forsythe Test:
Statistic: 64.17188598865032, p-value: 9.59502304853687e-14


# CATE len

In [18]:
i=8
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_len
Mann-Whitney U Test:
Statistic: 1261.0, p-value: 6.56433384736234e-20
Wilcoxon Signed-Rank Test:
Statistic: 670.0, p-value: 1.7930711033640755e-10
Kruskal-Wallis H Test:
Statistic: 83.46340895522383, p-value: 6.490631816767171e-20
Levene's Test:
Statistic: 9.245638818797852, p-value: 0.0026798315030190745
Brown-Forsythe Test:
Statistic: 9.245638818797852, p-value: 0.0026798315030190745


# ATE Cover SE

In [19]:
group1 = np.array(ps_bart_SE_empirical_ATE_cover)
group2 = np.array(bcf_SE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover SE
ATE Cover RMSE for ps-BART: 0.10042728712854887 and BCF: 0.3190982293902615
Mann-Whitney U Test:
Statistic: 209.0, p-value: 1.1927690422372805e-31
Wilcoxon Signed-Rank Test:
Statistic: 32.0, p-value: 1.0192401713884695e-17
Kruskal-Wallis H Test:
Statistic: 137.0502642273679, p-value: 1.1757103194880088e-31
Levene's Test:
Statistic: 43.13939953849398, p-value: 4.391418612195878e-10
Brown-Forsythe Test:
Statistic: 43.13939953849398, p-value: 4.391418612195878e-10


# ATE Cover AE

In [20]:
group1 = np.array(ps_bart_AE_empirical_ATE_cover)
group2 = np.array(bcf_AE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover AE
ATE Cover MAE for ps-BART: 0.08949999999999997 and BCF: 0.29407999999999995
Mann-Whitney U Test:
Statistic: 209.0, p-value: 1.1927690422372805e-31
Wilcoxon Signed-Rank Test:
Statistic: 23.0, p-value: 7.782249484736752e-18
Kruskal-Wallis H Test:
Statistic: 137.0502642273679, p-value: 1.1757103194880088e-31
Levene's Test:
Statistic: 36.29045732689987, p-value: 8.147990543278673e-09
Brown-Forsythe Test:
Statistic: 36.29045732689987, p-value: 8.147990543278673e-09


# CATE Cover SE

In [21]:
group1 = np.array(ps_bart_SE_theoretical_CATE_cover)
group2 = np.array(bcf_SE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover SE
CATE Cover RMSE for ps-BART: 0.0758987822995863 and BCF: 0.6072275959325959
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.561999346290125e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25384328646462, p-value: 2.5237972248690744e-34
Levene's Test:
Statistic: 121.45693735279097, p-value: 2.469542889309951e-22
Brown-Forsythe Test:
Statistic: 121.45693735279097, p-value: 2.469542889309951e-22


# CATE Cover AE

In [22]:
group1 = np.array(ps_bart_AE_theoretical_CATE_cover)
group2 = np.array(bcf_AE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover AE
CATE Cover MAE for ps-BART: 0.06570863999999994 and BCF: 0.6059052799999999
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.561999346290125e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25384328646462, p-value: 2.5237972248690744e-34
Levene's Test:
Statistic: 1.7508143256407818, p-value: 0.18730034996035078
Brown-Forsythe Test:
Statistic: 1.7508143256407818, p-value: 0.18730034996035078
